In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import entr
from drift_ml.datasets.bosch_cnc_machining.utils.utils import augment_xyz_samples
from drift_ml.datasets.bosch_cnc_machining.utils.dataloader import RawBoschCNCDataloader, STFTBoschCNCDataloader, NPYBoschCNCDataLoader

In [5]:
base_loader = NPYBoschCNCDataLoader(metadata_path="/home/tbiegel/nico_files/drift_ml/src/drift_ml/datasets/bosch_cnc_machining/extracted_features/metadata_ws4096.pkl")
base_loader.load_data(
    sample_data_x_path="/home/tbiegel/nico_files/drift_ml/src/drift_ml/datasets/bosch_cnc_machining/extracted_features/npy/sample_data_x_raw_ws4096.npy",
    sample_data_y_path="/home/tbiegel/nico_files/drift_ml/src/drift_ml/datasets/bosch_cnc_machining/extracted_features/npy/sample_data_y_raw_ws4096.npy"
    )
base_loader.generate_datasets_by_size(train_size=.5,val_size=.1,test_size=.4)

In [6]:
stft_base_loader = base_loader.get_windowed_samples_as_stft_dataloader(transform_fn=lambda x: x)
channel_means = np.mean(stft_base_loader.X_train, axis=(0, 2, 3), keepdims=True)
channel_stds = np.std(stft_base_loader.X_train, axis=(0, 2, 3), keepdims=True)

X_train_scaled = (stft_base_loader.X_train - channel_means) / channel_stds
X_val_scaled = (stft_base_loader.X_val - channel_means) / channel_stds
X_test_scaled = (stft_base_loader.X_test - channel_means) / channel_stds

  0%|          | 0/42978 [00:00<?, ?it/s]

In [17]:
from torch import tensor
from torchmetrics.classification import (
    BinaryAUROC,
    BinaryF1Score,
    BinaryAveragePrecision)
from drift_ml.datasets.bosch_cnc_machining.models.nnclassifier import NNEnsembleClassifier

ensemble = NNEnsembleClassifier()

In [ ]:
ensemble.fit(
    fit_args=[
        X_train_scaled, 
        stft_base_loader.y_train[:, np.newaxis], 
        X_val_scaled, 
        stft_base_loader.y_val[:, np.newaxis]
    ],
    fit_kwargs={
        "lrate":1e-2,
        "epochs":50
    }
)

In [19]:
auroc = BinaryAUROC()
auprc = BinaryAveragePrecision()
f1 = BinaryF1Score(threshold=0.5)

y, y_scores = ensemble.predict(X_test_scaled, return_scores=True)
y_scores = tensor(y_scores)
y_test = tensor(stft_base_loader.y_test[:, np.newaxis])

auroc_score = auroc(y_scores, y_test)
auprc_score = auprc(y_scores, y_test)
f1_score = f1(y_scores, y_test)

print(f"Test AUROC: {auroc_score:.3f}, AUPRC: {auprc_score:.3f}, F1: {f1_score:.3f}")

(5, 17269, 1)
Test AUROC: 0.649, AUPRC: 0.125, F1: 0.157


In [ ]:
shifts = np.arange(0, 100, 10)
f1_scores = []
auroc_scores = []
auprc_scores = []
avg_entropies = []

for shift in shifts:
    print(f"Evaluating {shift} degrees pitch shift")
    stft_loader = base_loader.get_windowed_samples_as_stft_dataloader(
        transform_fn=lambda x: augment_xyz_samples(x, pitch_deg=shift, yaw_deg=shift)
    )
    X_test_scaled = (stft_loader.X_test - channel_means) / channel_stds

    y, y_scores = ensemble.predict(X_test_scaled, return_scores=True)
    y_scores = tensor(y_scores)
    y_test = tensor(stft_loader.y_test[:, np.newaxis])

    auroc_score = auroc(y_scores, y_test)
    auprc_score = auprc(y_scores, y_test)
    f1_score = f1(y_scores, y_test)

    auroc_scores.append(auroc_scores)
    auprc_scores.append(auprc_score)
    f1_scores.append(f1_score)

    avg_entropy = np.mean(entr(y_scores.numpy()))
    avg_entropies.append(avg_entropy)

    print(f"Test AUROC: {auroc_score:.3f}, AUPRC: {auprc_score:.3f}, F1: {f1_score:.3f}, Average Entropy: {avg_entropy:.4f}")